# Structural and functional annotation of Gryllus longicercus

Szymon Szrajer 17-10-23

## 1. Prepare RNAseq evidence (BAM files).

### 1a. Download fastq.gz files

In [ ]:
%%bash 
# download fastq.gz files
cd /data/Glon_RNAseq/fastq
wget -O 00_fastq.zip https://www.dropbox.com/sh/qunrnjoguppui1p/AAAfHJROjaCNNcY-S_ME2pCPa\?dl\=1

In [ ]:
%%bash 
# unzip fastq.gz folder
cd /data/Glon_RNAseq/fastq
unzip 00_fastq.zip

### 1b. Run FastQC

In [ ]:
%%bash
cd /data/Glon_RNAseq/fastq
/data/Glon_RNAseq/software/FastQC/fastqc -o ../qc_precut -t 30 *gz

### 1c. Run Trim Galore!

In [ ]:
%%bash
cd /data/Glon_RNAseq/fastq
for r1_file in *_R1.fastq.gz; do
    # get_ the corresponding R2 file name by replacing "_R1" with "_R2"
    r2_file=${r1_file/_R1/_R2}
    
    /data/Glon_RNAseq/software/TrimGalore-0.6.10/trim_galore --paired -j 5 ${r1_file} ${r2_file}

    done

### 1d. Run FastQC


In [ ]:
%%bash
cd /data/Glon_RNAseq/fastq_trim
/data/Glon_RNAseq/software/FastQC/fastqc -o ../qc_postcut -t 30 *gz

### 1d. Run HISAT2!

In [ ]:
%%bash
cd /data/Glon_RNAseq/genomes
# build ht2 genome index
/data/Glon_RNAseq/software/hisat2-2.2.1/hisat2-build -p 30 Glon_curated.fasta Glon

In [ ]:
%%bash
cd /data/Glon_RNAseq/fastq_trim

#!/bin/bash
for fileR1 in *_R1_*gz; do
  # $file is the absolute path
  fileR2=$(echo $fileR1 | sed 's/R1_val_1/R2_val_2/')
  sampleR1=$(basename $fileR1 _R1_val_1.fq.gz)
  # print the results to stdout
  echo "$sampleR1,$fileR1"
  # here, the --dta flag is necessary to ensure that HISAT2 output can be used in the BRAKER3 pipeline.
  /data/Glon_RNAseq/software/hisat2-2.2.1/hisat2 --dta -q -p 20 -x /data/Glon_RNAseq/genomes/Glon -1 ${fileR1} -2 ${fileR2} \
    -S /data/Glon_RNAseq/bam/${sampleR1}.sam &> /data/Glon_RNAseq/bam/${sampleR1}_summary.txt
  /opt/samtools-1.17/samtools view -b /data/Glon_RNAseq/bam/${sampleR1}.sam  > /data/Glon_RNAseq/bam/${sampleR1}.bam
  rm /data/Glon_RNAseq/bam/${sampleR1}.sam
  done


## 2. Prepare protein evidence.

As for the BRAKER3 team's suggestion, the base protein set for this annotation was a set of 2.1Gb Arthropoda proteins from OrthoDB.

https://bioinf.uni-greifswald.de/bioinf/partitioned_odb11/

- [Gryllus bimaculatus *(25687)](https://www.ncbi.nlm.nih.gov/protein?term=txid6998%5BOrganism%5D#)
(((txid6998[Organism]) AND "Gryllus bimaculatus"[porgn:__txid6999])) AND PRJDB10609 (*PRJDB10609 is a BioProject)*
- [Gryllus pennsylvanicus x Gryllus firmus *(677)*](https://www.ncbi.nlm.nih.gov/protein?term=txid6998%5BOrganism%5D#)
- [Gryllus veletis *(98)](https://www.ncbi.nlm.nih.gov/protein?term=txid6998%5BOrganism%5D#)* (txid6998[Organism]) AND "Gryllus veletis"[porgn:__txid51040]
- [All other taxa *(185)](https://www.ncbi.nlm.nih.gov/protein?term=txid6998%5BOrganism%5D#)* (txid6998[Organism]) NOT "Gryllus veletis"[porgn:__txid51040] NOT "Gryllus pennsylvanicus"[porgn:__txid51074] NOT "Gryllus firmus"[porgn:__txid7000] NOT "Gryllus pennsylvanicus x Gryllus firmus"[porgn:__txid658220] NOT "Gryllus bimaculatus"[porgn:__txid6999]

In [ ]:
%%bash

cd /data/Glon_RNAseq/protein/supplementary # folder with downloaded NCBI sequences
cat *fasta > Gryllus_proteins.fasta

In [27]:
def replace_fasta_headers(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    with open(file_path, 'w') as file:
        for line in lines:
            if line.strip():  # check if line is not empty
                if line.startswith('>'):
                    header = line.strip().split()[0][1:]  # extract the first word without '>'
                    file.write(f'>{header} {header}\n')
                else:
                    file.write(line)


file_path = '/data/Glon_RNAseq/protein/Gryllus_proteins.fasta'
replace_fasta_headers(file_path)

Join the protein files together

In [28]:
%%bash
cd /data/Glon_RNAseq/protein
cat Arthropoda.fa Gryllus_proteins.fasta > proteinDB.fasta

## 3. Mask the genome.


### 1a. Using the RepBase library 

In [ ]:
# run on 10.10.4.125
!/usr/share/RepeatMasker/RepeatMasker -pa 20 -small -lib \
/home/cluster_user/2.Glon/RepeatMasking/libraries/RB.lib.fa /home/cluster_user/2.Glon/RepeatMasking/Glon_curated.fasta -gff \
-dir /home/cluster_user/2.Glon/RepeatMasking/soft_repbase/

### 1b. Using Gbim repeat library:

In [ ]:
# run on 10.10.4.125
!/usr/share/RepeatMasker/RepeatMasker -pa 20 -small -lib \
/home/cluster_user/2.Glon/RepeatMasking/libraries/Gbi_Rep_CombinedLibrary.lib.minlen50.nr.classified.filtered.fa \
/home/cluster_user/2.Glon/RepeatMasking/Glon_curated.fasta -gff \
-dir /home/cluster_user/2.Glon/RepeatMasking/soft_gbimlib/

### 2. Check masking with BUSCO

In [41]:
%%bash
cd /data/Glon_RNAseq/
docker pull ezlabgva/busco:v5.5.0_cv1
docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1

In [ ]:
%%bash
cd /data/Glon_RNAseq/

#docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1 busco -i /busco_wd/genomes/Glon_curated.fasta -o geno_a --out_path /busco_wd/buscos -m geno --augustus -l arthropoda -c 35 
#docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1 busco -i /busco_wd/genomes/Glon_curated.fasta -o geno_i --out_path /busco_wd/buscos -m geno --augustus -l insecta -c 35 
docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1 busco -i /busco_wd/masking/soft_repbase/ -o rep_a --out_path /busco_wd/buscos -m geno --augustus -l arthropoda -c 35 
docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1 busco -i /busco_wd/masking/soft_repbase/ -o rep_i --out_path /busco_wd/buscos -m geno --augustus -l insecta -c 35 
docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1 busco -i /busco_wd/masking/soft_gbimlib/ -o gbim_a --out_path /busco_wd/buscos -m geno --augustus -l arthropoda -c 35 
docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1 busco -i /busco_wd/masking/soft_gbimlib/ -o gbim_i --out_path /busco_wd/buscos -m geno --augustus -l insecta -c 35 

## 4. Run BRAKER3

### 1. BRAKER3 run with Docker

In [ ]:
%%bash

sudo docker run --user 1000:100 --rm -it \
    -v /home/cluster_user/data/Glon_RNAseq:/home/jovyan/ \
    teambraker/braker3:latest bash -c "braker.pl --genome=/home/jovyan/masking//Glon_soft_masked.fasta # <- \ 
    --workingdir=/home/jovyan/annotation --threads=40 --gff3 --prot_seq=/home/jovyan/protein/Gryllus_proteins.fasta \
    --bam=/home/jovyan/bam/FB12.bam,/home/jovyan/bam/FB13.bam,/home/jovyan/bam/FB4.bam,/home/jovyan/bam/FT11.bam,/home/jovyan/bam/FT12.bam,/home/jovyan/bam/FT13.bam,/home/jovyan/bam/FTA11.bam,/home/jovyan/bam/FTA12.bam,/home/jovyan/bam/FTA13.bam,/home/jovyan/bam/MB11.bam,/home/jovyan/bam/MB12.bam,/home/jovyan/bam/MB13.bam,/home/jovyan/bam/MT11.bam,/home/jovyan/bam/MT12.bam,/home/jovyan/bam/MT13.bam,/home/jovyan/bam/MTA11.bam,/home/jovyan/bam/MTA12.bam,/home/jovyan/bam/MTA13.bam \
    &> /home/jovyan/annotation/run.log"


### 2. Check Annotation's BUSCO score

In [ ]:
gffread

In [ ]:
docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1 busco -i /busco_wd/annotation -o prot_a --out_path /busco_wd/buscos -m protein --augustus -l arthropoda -c 35 
docker run -u $(id -u) -v $(pwd):/busco_wd ezlabgva/busco:v5.5.0_cv1 busco -i /busco_wd/annotation -o prot_i --out_path /busco_wd/buscos -m protein --augustus -l insecta -c 35

## 5. BLAST functional annotation

We first started with BLASTing against Swissprot Insecta, then continued with TrEMBL Insecta and tried to get a BLAST hit for the third time with Swissprot without taxonomy filtering.

In [ ]:
# Swissprot Insecta  9690 proteins
SI = "/home/cluster_user/functional2/libraries/revieved_insecta/revieved_swisssecta.fasta"
# TrEMBL Insecta 5683201 proteins
TI = "/home/cluster_user/functional2/libraries/unrevieved_insecta/unrevieved_insecta.fasta"
# Swissprot, no taxononomy filtering 569793 proteins
SA = "/home/cluster_user/functional2/libraries/revieved_all/uniprot_sprot.fasta"

In [ ]:
!makeblastdb -in $SI -dbtype prot -input_type fasta
!makeblastdb -in $TI -dbtype prot -input_type fasta
!makeblastdb -in $SA -dbtype prot -input_type fasta

In [ ]:
query = "/data/Glon_RNAseq"

In [ ]:
!blastp -num_threads 20 -query $query -db $SI -outfmt '6 std stitle' -out /home/cluster_user/functional2/libraries/revieved_insecta/SI_results.blastp -evalue 1e-6 \
-max_hsps 1 -max_target_seqs 1 

In [ ]:
!blastp -num_threads 20 -query $query -db $TI -outfmt '6 std stitle' -out /home/cluster_user/functional2/libraries/unrevieved_insecta/UI_results.blastp -evalue 1e-6 \
-max_hsps 1 -max_target_seqs 1 

In [ ]:
!blastp -num_threads 20 -query $query -db $SA -outfmt '6 std stitle' -out /home/cluster_user/functional2/libraries/revieved_all/SA_results.blastp -evalue 1e-6 \
-max_hsps 1 -max_target_seqs 1

In [ ]:
results_A = {}
results_B = {}
results_C = {}

with open("/home/cluster_user/functional2/libraries/revieved_insecta/SI_results.blastp", "r") as file_A:
    for line in file_A:
        fields = line.strip().split("\t")
        results_A[fields[0]] = line

with open("/home/cluster_user/functional2/libraries/unrevieved_insecta/UI_results.blastp", "r") as file_B:
    for line in file_B:
        fields = line.strip().split("\t")
        # check if the protein ID already exists in A results, if not, add it
        if fields[0] not in results_A:
            results_B[fields[0]] = line

with open("/home/cluster_user/functional2/libraries/revieved_all/SA_results.blastp", "r") as file_C:
    for line in file_C:
        fields = line.strip().split("\t")
        # check if the protein ID already exists in A or B results, if not, add it
        if fields[0] not in results_A and fields[0] not in results_B:
            results_C[fields[0]] = line

In [ ]:
print(len(results_A)) #9382
print(len(results_B)) #6k + 
print(len(results_C)) #10

In [ ]:
merged_results = {**results_A, **results_B, **results_C}

# write the merged results to an output file
with open("merged_results.txt", "w") as output_file:
    for result in merged_results.values():
        output_file.write(result)

## 6. InterProScan and Funannotate functional annotaion